This file includes all the steps involved in the creation of the project.

#### Libraries

In [19]:
import re
import time
import numpy as np
import pandas as pd
import os
import logging
from logging.handlers import RotatingFileHandler
from appium import webdriver
from appium.options.android import UiAutomator2Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

handler = RotatingFileHandler('app.log', maxBytes=5000, backupCount=2)
logging.basicConfig(
    handlers=[handler],
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.getLogger('selenium').setLevel(logging.WARNING)
logging.getLogger('urllib3').setLevel(logging.WARNING)
logging.getLogger('appium').setLevel(logging.WARNING)

#### Log Configuration

In [20]:
console_handler = logging.StreamHandler()
file_handler = logging.FileHandler('app.log')

logging.basicConfig(
    handlers=[console_handler, file_handler],
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

#### Initializing Driver

In [27]:
# Device information (keep these as environment variables)
DEVICE_NAME = os.getenv('DEVICE_NAME', 'R9ZWA07928P')
DEVICE_UDID = os.getenv('DEVICE_UDID', 'R9ZWA07928P')

desired_caps = {
    'platformName': 'Android',
    'deviceName': DEVICE_NAME,
    'udid': DEVICE_UDID,
    'platformVersion': '14',
    'appPackage': 'com.instagram.android',
    'appActivity': '.activity.MainTabActivity',
    'automationName': 'UiAutomator2',
    'noReset': True,
    'newCommandTimeout': 6000,  # Reduced timeout
    'adbExecTimeout': 20000,  # Increased timeout
    'autoGrantPermissions': True,
    'disableWindowAnimation': True,
    'unicodeKeyboard': True,
    'resetKeyboard': True,
    "appium:ensureWebviewsHavePages": True
}

try:
    # First, check if device is connected
    logging.info("Checking if device is connected...")
    import subprocess
    adb_devices = subprocess.check_output('adb devices').decode()
    logging.info(adb_devices)
    
    if DEVICE_UDID not in adb_devices:
        logging.error(f"Device {DEVICE_UDID} not found. Please connect your device and enable USB debugging.")
        raise Exception(f"Device {DEVICE_UDID} not found.")
    
    options = UiAutomator2Options().load_capabilities(caps=desired_caps)
    driver = webdriver.Remote('http://127.0.0.1:4723/wd/hub', options=options)
    logging.info("Driver initialized successfully")
    
    # Add a small delay to ensure the app is fully loaded
    import time
    time.sleep(5)
    
except Exception as e:
    logging.error(f"Error initializing driver: {str(e)}")
    raise

2025-03-22 20:51:42,283 - INFO - Checking if device is connected...
2025-03-22 20:51:42,396 - INFO - List of devices attached
R9ZWA07928P	device


2025-03-22 20:51:56,952 - INFO - Driver initialized successfully


#### X-PATHS

In [38]:
def get_dynamic_xpaths(usernames):
    base_xpaths = {
        'search_explore': '//android.widget.FrameLayout[@content-desc="Search and explore"]',
        'search_bar': '//android.widget.EditText[@resource-id="com.instagram.android:id/action_bar_search_edit_text"]',
        'search_bar_text': '//android.widget.EditText[@resource-id="com.instagram.android:id/action_bar_search_edit_text"]'
    }
    
    # Create dynamic XPATHs for each username
    dynamic_xpaths = {**base_xpaths}  # Copy base XPATHs
    dynamic_xpaths['search_results'] = {
        username: f'//android.widget.TextView[@resource-id="com.instagram.android:id/row_search_user_username" and @text="{username}"]'
    }
    
    return dynamic_xpaths
logging.info(f'X_PATHS: {x_paths}')

2025-03-22 21:12:02,793 - INFO - X_PATHS: {'search_explore': '//android.widget.FrameLayout[@content-desc="Search and explore"]', 'search_bar': '//android.widget.EditText[@resource-id="com.instagram.android:id/action_bar_search_edit_text"]', 'search_bar_text': '//android.widget.EditText[@resource-id="com.instagram.android:id/action_bar_search_edit_text"]', 'search_result': '//android.widget.TextView[@resource-id="com.instagram.android:id/row_search_user_username" and @text="None"]'}


#### Steps

In [39]:
def find_element(x_path_name):
    logging.info(f"Attempting to locate element: {x_path_name}")
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, x_paths[x_path_name])))
        logging.info(f"Element located and clicked: {x_path_name}")
        return element
    except Exception as e:
        logging.error(f"Failed to locate element: {x_path_name}. Error: {e}")
        raise

In [35]:
user_name = ['navneet_ks_', 'candidpavscene', 'sevainternal']
search_explore = find_element('search_explore')
search_explore.click()
time.sleep(1)
search_bar = find_element('search_bar')
search_bar.click()
time.sleep(1)
search_bar_text = find_element('search_bar_text')
search_bar_text.click()
search_bar_text.send_keys(user_name)

2025-03-22 21:02:15,872 - INFO - Attempting to locate element: search_explore
2025-03-22 21:02:16,584 - INFO - Element located and clicked: search_explore
2025-03-22 21:02:17,660 - INFO - Attempting to locate element: search_bar
2025-03-22 21:02:18,473 - INFO - Element located and clicked: search_bar
2025-03-22 21:02:20,427 - INFO - Attempting to locate element: search_bar_text
2025-03-22 21:02:31,055 - ERROR - Failed to locate element: search_bar_text. Error: Message: 
Stacktrace:
NoSuchElementError: An element could not be located on the page using the given search parameters.
    at AndroidUiautomator2Driver.findElOrEls (C:\Users\hp\AppData\Local\Programs\Appium Server GUI\resources\app\node_modules\appium\node_modules\appium-android-driver\lib\commands\find.js:75:11)
    at runMicrotasks (<anonymous>)
    at processTicksAndRejections (internal/process/task_queues.js:97:5)
    at AndroidUiautomator2Driver.findElOrElsWithProcessing (C:\Users\hp\AppData\Local\Programs\Appium Server GU

TimeoutException: Message: 
Stacktrace:
NoSuchElementError: An element could not be located on the page using the given search parameters.
    at AndroidUiautomator2Driver.findElOrEls (C:\Users\hp\AppData\Local\Programs\Appium Server GUI\resources\app\node_modules\appium\node_modules\appium-android-driver\lib\commands\find.js:75:11)
    at runMicrotasks (<anonymous>)
    at processTicksAndRejections (internal/process/task_queues.js:97:5)
    at AndroidUiautomator2Driver.findElOrElsWithProcessing (C:\Users\hp\AppData\Local\Programs\Appium Server GUI\resources\app\node_modules\appium\node_modules\appium-base-driver\lib\basedriver\commands\find.js:33:12)
    at AndroidUiautomator2Driver.findElement (C:\Users\hp\AppData\Local\Programs\Appium Server GUI\resources\app\node_modules\appium\node_modules\appium-base-driver\lib\basedriver\commands\find.js:53:10)